## Import Libraries and Load Data

In [1]:
import tensorflow as tf
import numpy as np
import cv2

In [2]:
from google.colab.patches import cv2_imshow

In [9]:
(X_train, y_train) , (X_test, y_test) = tf.keras.datasets.mnist.load_data(path='mnist.npz')

In [4]:
cv2_imshow(X_train[0])

In [5]:
X_train.shape

(60000, 28, 28)

## Reduce image size to 20x20 for BCS

In [11]:
X_train_red = np.zeros((60000, 20, 20))
for i in range(len(X_train)):
  X_train_red[i] = cv2.resize(X_train[i], (20,20))

In [12]:
cv2_imshow(X_train_red[0])

In [13]:
X_train_red.shape

(60000, 20, 20)

## Define Architecture

In [22]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.losses import MSE
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential

In [24]:
model = Sequential([
    Input(shape = (20,20)),
    Flatten(),
    Dense(400, activation='relu'), #20x20 image vector flattened [B^2 x 1 ] 
    Dense(40, activation = 'relu'), # B^2 x R where R = 0.1 (sampling rate)
    Dense(1600, activation = 'relu'), #B^2 x T  where T = 4 (redundancy rate)
    Dense(1600, activation = 'relu'), # we use total 2 reconstruction layers
    Dense(400, activation = 'relu') #final output = B^2 x 1
])

In [25]:
model.compile(optimizer=Adagrad(learning_rate=0.005), loss= 'mse')

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 400)               160400    
_________________________________________________________________
dense_11 (Dense)             (None, 40)                16040     
_________________________________________________________________
dense_12 (Dense)             (None, 1600)              65600     
_________________________________________________________________
dense_13 (Dense)             (None, 1600)              2561600   
_________________________________________________________________
dense_14 (Dense)             (None, 400)               640400    
Total params: 3,444,040
Trainable params: 3,444,040
Non-trainable params: 0
____________________________________________

In [28]:
y_train = X_train_red.reshape(60000, 400)

In [30]:
model.fit(X_train_red, y_train, epochs=100, batch_size=16)

Epoch 1/100
3750/3750 [==============================] - 99s 26ms/step - loss: 594.9590
Epoch 2/100
3750/3750 [==============================] - 98s 26ms/step - loss: 551.9876
Epoch 3/100
3750/3750 [==============================] - 99s 26ms/step - loss: 530.3635
Epoch 4/100
3750/3750 [==============================] - 98s 26ms/step - loss: 516.1763
Epoch 5/100
3750/3750 [==============================] - 98s 26ms/step - loss: 505.5542
Epoch 6/100
3750/3750 [==============================] - 99s 27ms/step - loss: 497.2821
Epoch 7/100
3750/3750 [==============================] - 98s 26ms/step - loss: 490.5014
Epoch 8/100
3750/3750 [==============================] - 98s 26ms/step - loss: 484.7824
Epoch 9/100
3750/3750 [==============================] - 99s 26ms/step - loss: 479.8696
Epoch 10/100
3750/3750 [==============================] - 98s 26ms/step - loss: 475.6161
Epoch 11/100
3750/3750 [==============================] - 98s 26ms/step - loss: 471.7987
Epoch 12/100
3750/3750 [======

In [31]:
y0 = model.output[0]

In [37]:
X_train_red[0].shape

(20, 20)

In [38]:
Y = model.predict(X_train_red)

In [45]:
index = 1
reconstructed_y = Y[index].reshape(20,20)

In [46]:
cv2_imshow(X_train_red[index])

In [47]:
cv2_imshow(reconstructed_y)

In [89]:
X_test_red = np.zeros((60000, 20, 20))
for i in range(len(X_test)):
  X_test_red[i] = cv2.resize(X_test[i], (20,20))

In [90]:
Y_check = model.predict(X_test_red)

In [91]:
index = 1
reconstructed_y = Y_check[index].reshape(20,20)

In [92]:
cv2_imshow(X_test_red[index])

In [93]:
cv2_imshow(reconstructed_y)

## ReconNet

*Now that we have a trained compressive senisng matrix,
   we can use that matrix to get compressed measurements and
   this time we will use ReconNet which utilizes CNNs to 
   reconstruct original images instead of Fully-Connected layers
   as in previous model.*

In [66]:
from tensorflow.keras.layers import Conv2D, Dense, AveragePooling2D, Input, Reshape
from tensorflow.keras.models import Model

In [72]:
def ReconNet(cs_measurements):
  x = Dense(400, activation = 'relu')(cs_measurements)
  x = Reshape((20, 20, 1))(x)
  x = Conv2D(64, (11,11), padding='same', activation='relu',
             kernel_initializer='random_normal')(x)
  x = Conv2D(32, (1,1), padding='same', activation='relu',
             kernel_initializer='random_normal')(x)
  x = Conv2D(1, (7,7), padding='same', activation='relu',
             kernel_initializer='random_normal')(x)
  x = Conv2D(64, (11,11), padding='same', activation='relu',
             kernel_initializer='random_normal')(x)
  x = Conv2D(32, (1,1), padding='same', activation='relu',
             kernel_initializer='random_normal')(x)
  output = Conv2D(1, (7,7), padding='same', activation='relu',
             kernel_initializer='random_normal')(x)
  return output

In [82]:
#get weights of the first layer of previous trained model
w = model.layers[2].get_weights()

In [78]:
first_layer = Input(shape = (20, 20))
second_layer = Flatten()(first_layer)
#third_layer = Dense(400, activation = 'relu')(second_layer)
cs_layer = Dense(40, activation = 'relu')(second_layer)
recon_model = ReconNet(cs_layer)

In [79]:
model2 = Model(inputs = first_layer, outputs = recon_model)

In [80]:
model2.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 20, 20)]          0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 40)                16040     
_________________________________________________________________
dense_28 (Dense)             (None, 400)               16400     
_________________________________________________________________
reshape_2 (Reshape)          (None, 20, 20, 1)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 20, 20, 64)        7808      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 20, 20, 32)        2080

In [83]:
model2.layers[2].set_weights(w)

In [84]:
model2.layers[2].trainable = False

In [85]:
model2.compile(optimizer=Adagrad(learning_rate=0.005), loss= 'mse')

In [87]:
y_train_recon = X_train_red.reshape(60000,20,20,1)

In [94]:
history = model2.fit(X_train_red, y_train_recon, epochs = 20)

Epoch 1/20
1875/1875 [==============================] - 483s 258ms/step - loss: 1444.8824
Epoch 2/20
1875/1875 [==============================] - 483s 257ms/step - loss: 1321.6982
Epoch 3/20
1875/1875 [==============================] - 483s 258ms/step - loss: 1253.9779
Epoch 4/20
1875/1875 [==============================] - 485s 259ms/step - loss: 1208.7560
Epoch 5/20
1875/1875 [==============================] - 484s 258ms/step - loss: 1175.4229
Epoch 6/20
1875/1875 [==============================] - 482s 257ms/step - loss: 1149.6223
Epoch 7/20
1875/1875 [==============================] - 482s 257ms/step - loss: 1128.5186
Epoch 8/20
1875/1875 [==============================] - 482s 257ms/step - loss: 1111.2919
Epoch 9/20
1875/1875 [==============================] - 482s 257ms/step - loss: 1096.1576
Epoch 10/20
1875/1875 [==============================] - 482s 257ms/step - loss: 1083.3235
Epoch 11/20
1875/1875 [==============================] - 482s 257ms/step - loss: 1071.9188
Epoch 12